In [1]:
# Execute extenttion to reload files in project dir
%load_ext autoreload
%autoreload 2

In [2]:
# Add link to project root
import sys, os
sys.path.append(os.path.abspath('..'))

In [89]:
from source import utils
import uuid

### Документация к АПИ
https://bybit-exchange.github.io/docs/v5/intro

In [27]:
# Получаем сессию по кредам
from pybit.unified_trading import HTTP
session = HTTP(
    # testnet=True,
    api_key=utils.get_secrets('BYBIT_BOT_API'),
    api_secret=utils.get_secrets('BYBIT_BOT_SECRET'),
)

In [39]:
# Получанние серверного времени
session.get_server_time()['result']

{'timeSecond': '1709400948', 'timeNano': '1709400948960740335'}

{'list': [{'accountType': 'spot',
   'accountIMRate': '',
   'accountMMRate': '',
   'accountLTV': '',
   'totalEquity': '',
   'totalWalletBalance': '',
   'totalMarginBalance': '',
   'totalAvailableBalance': '',
   'totalPerpUPL': '',
   'totalInitialMargin': '',
   'totalMaintenanceMargin': '',
   'coin': [{'coin': 'USDT',
     'equity': '',
     'usdValue': '',
     'walletBalance': '49.4459',
     'free': '49.4459',
     'locked': '0',
     'availableToWithdraw': '',
     'availableToBorrow': '',
     'borrowAmount': '',
     'accruedInterest': '',
     'totalOrderIM': '',
     'totalPositionIM': '',
     'totalPositionMM': '',
     'unrealisedPnl': '',
     'cumRealisedPnl': ''}]}]}

### Пробуем провести транзакцию

In [131]:
session.get_instruments_info(
    category="linear",
    symbol="BTCUSDT",
)

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'contractType': 'LinearPerpetual',
    'status': 'Trading',
    'baseCoin': 'BTC',
    'quoteCoin': 'USDT',
    'launchTime': '1584230400000',
    'deliveryTime': '0',
    'deliveryFeeRate': '',
    'priceScale': '2',
    'leverageFilter': {'minLeverage': '1',
     'maxLeverage': '100.00',
     'leverageStep': '0.01'},
    'priceFilter': {'minPrice': '0.10',
     'maxPrice': '199999.80',
     'tickSize': '0.10'},
    'lotSizeFilter': {'maxOrderQty': '155.000',
     'minOrderQty': '0.001',
     'qtyStep': '0.001',
     'postOnlyMaxOrderQty': '155.000',
     'maxMktOrderQty': '100.000'},
    'unifiedMarginTrade': True,
    'fundingInterval': 480,
    'settleCoin': 'USDT',
    'copyTrading': 'both',
    'upperFundingRate': '0.00375',
    'lowerFundingRate': '-0.00375'}],
  'nextPageCursor': ''},
 'retExtInfo': {},
 'time': 1709410516753}

In [70]:
# Получить баланс на спот аккаунте по валюте
usdt_balance = session.get_wallet_balance(
    accountType="spot",
    coin="USDT",
)['result']['list'][0]['coin'][0]['walletBalance']

print(f'Your USDT balance is {usdt_balance}')

Your USDT balance is 49.4459


In [78]:
TIKER_TO_BUY = 'MANTAUSDT'
ORDER_DEPO = 5 # USDT

last_price = float(session.get_tickers(category="spot", symbol=TIKER_TO_BUY,)['result']['list'][0]['lastPrice'])
order_qty = round(ORDER_VALUE / last_price, 5)
print(f"last proce of {TIKER_TO_BUY} is {last_price} USDT")
print(f'for order volume of {ORDER_VALUE} you can buy {order_qty} {TIKER_TO_BUY}')

last proce of MANTAUSDT is 2.8752 USDT
for order volume of 5 you can buy 1.73901 MANTAUSDT


In [82]:
# Покупаем на споте 

# session.place_order(
#     category="spot", # spot, linear, inverse, option
#     symbol=TIKER_TO_BUY,
#     side="Buy", # Buy, Sell
#     orderType="Market", # Market | Limit
#     qty=ORDER_DEPO - order_qty, # in USDT
#     # price="15600", # Market order will ignore this field
#     # timeInForce="PostOnly",
#     # orderLinkId=str(uuid.uuid4()),
#     isLeverage=0,
#     orderFilter="Order",
# )

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'orderId': '1633001775002849536',
  'orderLinkId': 'spot-test-postonly_'},
 'retExtInfo': {},
 'time': 1709404981437}

In [93]:
# Получить баланс на аккаунте фьючей по валюте
usdt_balance = session.get_wallet_balance(
    accountType="UNIFIED",
    coin="USDT",
)['result']['list'][0]['coin'][0]['walletBalance']

print(f'Your USDT balance is {usdt_balance}')

Your USDT balance is 44.46983


In [118]:
# Поставить плечо для тикера
session.set_leverage(
    category="linear",
    symbol=TIKER_TO_BUY,
    buyLeverage="8",
    sellLeverage="8",
)

{'retCode': 0,
 'retMsg': 'OK',
 'result': {},
 'retExtInfo': {},
 'time': 1709409088980}

In [122]:
# Покупаем фьючи 
session.place_order(
    category="linear", # spot, linear, inverse, option
    symbol=TIKER_TO_BUY,
    side="Buy", # Buy, Sell
    orderType="Market", # Market | Limit
    qty=5, # in USDT
    # price="15600", # Market order will ignore this field
    # timeInForce="PostOnly",
    # orderLinkId=str(uuid.uuid4()),
    # reduceOnly=True,
    isLeverage=1,
    # orderIv=0.05,
    orderFilter="Order",
)

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'orderId': '6c10aa88-830c-4eb6-8cab-cb60d0fa02c6',
  'orderLinkId': ''},
 'retExtInfo': {},
 'time': 1709409563067}

In [119]:
session.get_open_orders(
    category="linear",
    symbol=TIKER_TO_BUY,
    openOnly=0,
    limit=1,
)

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'nextPageCursor': '', 'category': 'linear', 'list': []},
 'retExtInfo': {},
 'time': 1709409295409}